# Here Comes the Sun

Code for showing were batteries are in CAISO, and how they grew over time

In [11]:
#import modules 
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
import hvplot
import hvplot.pandas
import pathlib
import os
from glob import glob
import geopandas as gpd
import requests




In [13]:
#set up data directory 

map_dir = os.path.join(
    pathlib.Path.home(),
    'code-projects',
    'generation-analysis',
    'map-data'
    )
os.makedirs(map_dir, exist_ok=True)

map_data = os.path.join (map_dir, 'CA-power-plant-data.geojson')

In [35]:
# retrieve power plant data 

# Set up URL 
caiso_url = (
    "https://services3.arcgis.com/"
    "bWPjFyq029ChCGur/arcgis/rest/services"
    "/Power_Plant/FeatureServer"
    "/0/query?outFields=*&where=1%3D1&f=geojson"
)

#download data
if not os.path.exists(map_data):
   #Get File with requests
   response = requests.get(caiso_url, stream=True)
   #save URL Content to file
   with open(map_data, "wb") as file:
      #use Chunk to avoid memory overload 
      for chunk in response.iter_content(chunk_size=8192):
         file.write(chunk)

caiso_gdf = gpd.read_file(map_data)

caiso_gdf['StartDate'] = pd.to_datetime(caiso_gdf['StartDate'], unit= 'ms')

caiso_gdf


,OBJECTID,CECPlantID,PlantName,Retired_Plant,OperatorCompanyID,County,Capacity_Latest,Units,PriEnergySource,StartDate,CEC_Jurisdictional,geometry
0,1865,S0335,Corcoran 2 Solar LLC CED,0,RWE Clean Energy,Kings,19.80,1,SUN,2015-06-10,0,POINT (-119.56791 36.13717)
1,1866,S0520,Corcoran 3 Solar,0,RWE Clean Energy,Kings,20.00,Unit 1,SUN,2016-02-11,0,POINT (-119.57972 36.14432)
2,1867,C0007,Hanford - Retired October 2011,1,Hanford LP,Kings,24.00,GEN 1,PC,1990-09-01,0,POINT (-119.64840 36.26965)
3,1868,G0832,Hanford Energy Park Peaker,0,"MRP San Joaquin Energy, LLC",Kings,92.00,"1, 2",NG,2001-09-01,1,POINT (-119.64745 36.27031)
4,1869,S0608,Exeter Solar,0,Altus Power,Tulare,3.50,ES,SUN,2014-02-12,0,POINT (-119.12834 36.26630)
5,1870,S0609,Lindsay Solar,0,Altus Power,Tulare,4.00,LS,SUN,2014-01-15,0,POINT (-119.13401 36.21178)
6,1871,S9325,ImModo California 1 LLC_(East Orosi 1),0,Not Available,Tulare,1.50,Unit 1,SUN,2013-12-31,0,POINT (-119.03996 36.10372)
7,1872,S9326,ImModo California 1 LLC_(East Orosi 2),0,Not Available,Tulare,1.50,Unit 1,SUN,2013-12-31,0,POINT (-119.03880 36.10162)
8,1873,S9327,ImModo California 1 LLC_(East Orosi 3),0,Not Available,Tulare,1.50,Unit 1,SUN,2013-12-31,0,POINT (-119.03736 36.09974)
9,1874,S0151,Porterville Solar,0,Tulare PV I LLC,Tulare,3.50,PS,SUN,2014-01-13,0,POINT (-119.03968 36.10279)


In [36]:
# Organize battery geodata frame 

gdf_filtered = caiso_gdf[
            (caiso_gdf['PriEnergySource'] == 'SUN') &
            (caiso_gdf['StartDate'] != 'NaN') &
            (caiso_gdf['Retired_Plant'] != '1')
             ]

solar_gdf = gdf_filtered[['PlantName','Capacity_Latest', 'PriEnergySource', 'StartDate', 'geometry']]

solar_gdf = solar_gdf.dropna()


display(solar_gdf)

,PlantName,Capacity_Latest,PriEnergySource,StartDate,geometry
0,Corcoran 2 Solar LLC CED,19.80,SUN,2015-06-10,POINT (-119.56791 36.13717)
1,Corcoran 3 Solar,20.00,SUN,2016-02-11,POINT (-119.57972 36.14432)
4,Exeter Solar,3.50,SUN,2014-02-12,POINT (-119.12834 36.26630)
5,Lindsay Solar,4.00,SUN,2014-01-15,POINT (-119.13401 36.21178)
6,ImModo California 1 LLC_(East Orosi 1),1.50,SUN,2013-12-31,POINT (-119.03996 36.10372)
7,ImModo California 1 LLC_(East Orosi 2),1.50,SUN,2013-12-31,POINT (-119.03880 36.10162)
8,ImModo California 1 LLC_(East Orosi 3),1.50,SUN,2013-12-31,POINT (-119.03736 36.09974)
9,Porterville Solar,3.50,SUN,2014-01-13,POINT (-119.03968 36.10279)
10,Sequoia PV3,3.00,SUN,2014-08-01,POINT (-119.03678 36.09883)
14,CA Dept of Correctionns Salinas (SVSP),2.00,SUN,2013-01-01,POINT (-121.36947 36.47229)


In [37]:
solar_gdf.hvplot(geo=True, tiles=True)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]